<a href="https://colab.research.google.com/github/Jlahitette/Python/blob/main/fonctions_projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
cd


/content


In [57]:
import re
PATH='/content/drive/MyDrive/Notebooks_python/'
#PATH='./LU2SV483/Projet/'


############# ETAPE 1 ##################
def longueur(sequence):
	'''str-->int
	preconditions :none
	renvoie la longueur de la sequence passée en argument'''

	return len(sequence)

def taux_GC(sequence):
	'''str-->float
	preconditions : none
	renvoie le taux de G et C dans la sequence passée en argument'''

	#somme:int;nombre de G ou C
	somme=0
	#i:int;variable parcourrant la sequence
	for i in range(longueur(sequence)):
		if sequence[i]=='G' or sequence[i]=='C':
			somme=somme+1
	#taux:float;taux de G/C
	taux=round(somme*100/len(sequence),2)

	return taux

def sequence_inverse(sequence):
	'''str-->str
	preconditions: none
	renvoie la sequence inverse-complementaire de la sequence passée en argument'''
	#inverse:str;sequence inverse
	inverse=''

	#i:int;variable parcourrant la sequence
	for i in range(longueur(sequence)):
			if sequence[i]=='A':
				inverse=inverse+'T'
			elif sequence[i]=='T':
				inverse=inverse+'A'
			elif sequence[i]=='C':
				inverse=inverse+'G'
			else:
				inverse=inverse+'C'

	return inverse

def ATG_et_stop(sequence):
	'''str-->List[int]*List[int]
	preconditions:none
	renvoie une liste avec les positions de tout les codons stop et une autre avec les positions des codons start sur les deux brins'''

	#positions_start:	list[int] ; liste de toutes les positions des codons ATG
	positions_start=[]
	#positions_stop:	list[int] ; liste de toutes les positions des codons stop
	positions_stop=[]

	#i:int;variable parcourrant la sequence
	for i in range(longueur(sequence)):
		if sequence[i]=='A' and sequence[i+1]=='T' and sequence[i+2]=='G':				#codon ATG
			positions_start.append(i)
		if sequence[i]=='T':
			if sequence[i+1]=='A' and (sequence[i+2]=='A' or sequence[i+2]=='G'):		#codons TAA et TAG
				positions_stop.append(i)
			elif sequence[i+1]=='G' and sequence[i+2]=='A':								#codon TGA
				positions_stop.append(i)


	return positions_start,positions_stop

def app_sub_indel(seq1,seq2):
	'''str*str-->int*int*int
	preconditions:seq1 et seq2 ont la meme longueur
	renvoie le nombre d'appariements, de substitutions et le nombre d'indel'''

	#nb_app:int; nombre d'appariements
	#nb_sub:int;nombre de substitutions
	#nb_indel:int;nombre d'indels

	nb_app=nb_sub=nb_indel=0

	#i;int:variable parcourrant seq1
	#j;int:variable parcourrant seq2
	i=j=0
	while i<len(seq1):
		if seq1[i]=='-':
			nb_indel=nb_indel+1
		elif seq2[j]=='-':
			nb_indel=nb_indel+1
		else:
			if seq1[i]==seq2[j]:
				nb_app=nb_app+1
			if seq1[i]!=seq2[j]:
				nb_sub=nb_sub+1
		i=i+1
		j=j+1

	return nb_app,nb_sub,nb_indel

def distance_edition(seq1,seq2):
	'''str*str-->int
	preconditions:
	renvoie la distance d'edition entre la seq1 et la seq2'''

	#x:int:variable stockant le resultats (3 valeurs) de la fonction app_sub_indel
	x=app_sub_indel(seq1,seq2)

	#distance_edition:int;la distance d'edition
	distance_edition=x[1]+x[2]

	return distance_edition


def score_alignement(seq1,seq2,Wa,Ws,Wi):
	'''str*str*int*int*int-->int
	preconditions : Wa > Wi et Ws
	renvoie la somme des poids correspondant a chaque position pour calculer le score d'alignement entre seq1 et seq2'''

	#x:int:variable stockant le resultats (3 valeurs) de la fonction app_sub_indel
	x=app_sub_indel(seq1,seq2)

	#score:int;score d'alignement entre seq1 et seq2
	score=(Wa*x[0])+(Ws*x[1])+(Wi*x[2])

	return score


def taux_identite(seq1,seq2):
	'''str*str-->float
	preconditions: seq1 et seq2 ont la meme longueur
	renvoie le taux de ressemblance entre seq1 et seq2'''

	#x:int:variable stockant le resultats (3 valeurs) de la fonction app_sub_indel
	x=app_sub_indel(seq1,seq2)

	longueur=len(seq1)

	taux=round(x[0]*100/longueur,2)

	return taux

############# ETAPE 2 ##################

def lecture(fichier):
	'''str-->List[str]
	preconditions: none
	renvoie une liste du fichier passé en argument'''

	flux_entree=open(PATH+fichier,'r')
	fichier=flux_entree.readlines()
	flux_entree.close()

	return fichier

def informations(liste):
	'''List[str]--->List[str]
	preconditions :
	renvoie un resume des informations cles pour chaque CDS sous forme de liste'''

	CDS_true=False
	liste_principal=[]
	id_seq=''
	esp_seq=''
	debut=''
	fin=''
	fonction_seq=''
	uniprot_seq=''
	split3=''
	split3_bis=''
	split3_tri=''

	for ligne in liste :
		ligne.rstrip('\n')


		if re.search('^ID *',ligne):
			split1=ligne.split(';')
			split1_bis=split1[0].split('   ')
			id_seq_temp=split1_bis[1]

		if re.search('^OS *',ligne):
			split2=ligne.split()
			esp_seq_temp=split2[1]+' '+split2[2]


		if re.search('FT *CDS *[0-9]*\.\.[0-9]*',ligne):
			CDS_true=True
			if CDS_true==True:
				liste_principal.append([id_seq,esp_seq,debut,fin,fonction_seq,uniprot_seq])
				id_seq=id_seq_temp
				esp_seq=esp_seq_temp
			if re.search('FT *CDS *[0-9]*\.\.[0-9]*',ligne) and re.search('^[^><]*$',ligne):
				split3=ligne.split('..')
				split3_bis=split3[0].split()
				split3_tri=split3[1].split('\n')
				debut=split3_bis[2]
				fin=split3_tri[0]


		if re.search('^FT */product',ligne):
			split4=ligne.split()
			split4_bis=split4[1].split('"')
			fonction_seq=split4_bis[1]


		if re.search('^FT */db_xref="UniProtKB/TrEMBL',ligne):
			split5=ligne.split(':')
			split5_bis=split5[1].split('"')
			uniprot_seq=split5_bis[0]

	return liste_principal

def tableau(liste):
	'''list[list]--> ??
	precondtions :
	renvoie un tableau ordonné a partir des informaations passés en argument en foramt de liste de listes'''

	print('{:^20s} {:^40s} {:^30s} {:^60s} {:^10s}'.format('ID sequence', 'Espece', 'Debut - Fin', 'Fonction', 'Uniprot ID'))

	for i in range(len(liste)):
		print('{:<20s} {:<40s} {:>10s} -    {:^10s} {:<60s} {:<10s}'.format(liste[i][0],liste[i][1],liste[i][2],liste[i][3],liste[i][4],liste[i][5]))

	return

############# programme principal ########################
###Etape 1
#question 1
seq_exemple='ATCGTATGAAUCTATGUAACC'

r=longueur(seq_exemple)
#print('La longueur de la sequence est de',r,'nucléotides')

s=taux_GC(seq_exemple)
#print('Le taux de G/C dans la sequence est:',s,'%')

t=sequence_inverse(seq_exemple)
#print('La sequence complementaire inversé est:',t)


u=ATG_et_stop(seq_exemple)
#print('Les positions des codons start de la sequence sont :',u[0],'et celles des codons stop sont :',u[1])

#seq_comp:str;liste complementaire inverse de la sequence
seq_comp=sequence_inverse(seq_exemple)
v=ATG_et_stop(seq_comp)
#print('Les positions des codons start de la sequence complementaire sont :',v[0],'et celles des codons stop sont :',v[1])


#question 2

seq_ex1='CA-CGTGCTGACCCAACC'
seq_ex2='CAGCGCGCTGG--CAGCC'
w=app_sub_indel(seq_ex1,seq_ex2)
#print("Le nombre d'appariements est de",w[0],", le nombre de substitutions est de",w[1],"et le nombre d'indels est de",w[2])

x=distance_edition(seq_ex1,seq_ex2)
#print("La distance d'edition entre la seq1 et seq2 est de",x)

y=score_alignement(seq_ex1,seq_ex2,2,-1,-2)
#print("Le score d'alignement entre seq1 et seq2 est:",y)

z=taux_identite(seq_ex1,seq_ex2)
#print("Le taux d'identité entre seq1 et seq2 est de",z,'%')

###Etape 2
#question 1
liste=lecture('Bartonella.dat')

a=informations(liste)
#print(len(a))
#print(a)

b=tableau(a)


    ID sequence                       Espece                           Debut - Fin                                     Fonction                           Uniprot ID
                                                                         -               hypothetical                                                 J1JPZ3    
AILX01000001         Bartonella washoeensis                         9670 -      10536    orotidine                                                    J0ZFP0    
AILX01000002         Bartonella washoeensis                          198 -       308     hypothetical                                                 J0QUH6    
AILX01000002         Bartonella washoeensis                          335 -       532     hypothetical                                                 J1JQV8    
AILX01000002         Bartonella washoeensis                          776 -       2266    histidyl-tRNA                                                J0QMA8    
AILX01000002         Bartonell